In [1]:
import pandas
import psycopg2
import inspect
def unique_signature(f, *args, **kwargs):
    sig=inspect.signature(f)
    b=sig.bind(*args, **kwargs)
    b.apply_defaults()
    return b

# Background

Python and other C-like languages use a mixture of Infix and Prefix notation:

```python
a = pandas.read_sql(sql, conn) 
```

__Infix part:__

```
= - a
  \ pandas.read_sql(...)
```

__Prefix part:__
```
pandas.read_sql - sql
                \ conn
```

My goal is to create a Prefix IR between Python and similar languages which could then be trivially converted to SQL.

## Why Prefix IR:


 - The prefix IR is still valid Python, so we can evaluate it in Python (and get SQL in return):
 
A simple example of Infix to Prefix notation:

```python
ASSIGN(target=a, source=read_sql(sql,conn)) # not part of the IR
```
 - SQL looks Prefix-ish to me:
 ```SQL
SELECT A.data, B.data
FROM B
    INNER JOIN A ON B.CustomerID=A.CustomerID; *optional
WHERE *optional
ORDER BY *optinal
```

I can reconstruct the above SQL query (using nested queries) if I have the following functions:
```python
>>> PROJ(from_f, cols_f)
'SELECT '+cols_f()+' FROM '+from_f()

>>> SEL(from_f, cond_f) 
'SELECT * FROM '+from_f()+' WHERE '+cond_f()

>>> ORDER_BY(from_f, order_by_f) 
'SELECT * FROM '+from_f()+' ORDER BY '+order_by_f()

>>> JOIN(from_f1, from_f2, join_type, cond_f)
'SELECT * FROM '+from_f1()+' '+join_type()+' JOIN '+from_f2()+' ON '+cond_f()

>>> MAX(from_f, cols_f, group_by=lambda:'') # not 
'SELECT MAX('+cols_f()+') FROM '+from_f()+' GROUP BY ('+group_by()+')'
# note: GROUP BY () means no GROUP BY

>>> MIN...

>>> FINAL(from_f)
from_f()+';'

# and other functions
```

Therefore I need to create a mapping from a Python API (e.g. Pandas) to the functions above

In [2]:
import yaml
yaspec="""
pandas.DataFrame.append:
    args: [$with]
    kwargs: {}
    inplace: False
    maps: UNION($self, $with)
    
pandas.DataFrame.append:
    args: [$with, $with2]
    kwargs: {}
    inplace: False
    maps: UNION($self, $with)

pandas.Series.max:
    args: []
    kwargs: {}
    inplace: False
    maps: MAX($self)

pandas.read_sql:
    args: [$sql, $conn]
    kwargs: {}
    inplace: False
    maps: FROM($sql, $conn)
"""
pyspec = yaml.load(yaspec, Loader=yaml.FullLoader);pyspec

{'pandas.DataFrame.append': {'args': ['$with'],
  'kwargs': {},
  'inplace': False,
  'maps': 'UNION($self, $with)'},
 'pandas.Series.max': {'args': [],
  'kwargs': {},
  'inplace': False,
  'maps': 'MAX($self)'},
 'pandas.read_sql': {'args': ['$sql', '$conn'],
  'kwargs': {},
  'inplace': False,
  'maps': 'FROM($sql, $conn)'}}

# Theory
### mapping + code -> optimized code

In [9]:
What about default argument handling?

Object `handling` not found.


In [ ]:
What about default argument handling

In [3]:
unique_signature(pandas.read_sql, 'SELECT...', 'conn')

<BoundArguments (sql='SELECT...', con='conn', index_col=None, coerce_float=True, params=None, parse_dates=None, columns=None, chunksize=None)>

In [8]:
unique_signature(pandas.read_sql, 'SELECT...', 'conn', None, True, None, None)

<BoundArguments (sql='SELECT...', con='conn', index_col=None, coerce_float=True, params=None, parse_dates=None, columns=None, chunksize=None)>

In [ ]:
'host=localhost dbname=tpch user=p2d2 password=p2d2'